In [2]:
!pip install tensorflow==2.15.0
!pip install tensorflow-probability==0.23.0


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 68.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 57.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 100.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.3 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.16.0
    Uninstalling wrapt-1.16.0:
      Successfully uninstalled wrapt-1.16.0
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.4.0
    Uninstalling ml-dtypes-0.4.0:
      Successfully uninstalled ml-dtypes-0.4.0
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1
  Attempting uninstall: tensorboard
    Found existing installatio

In [1]:
import shutil
import zipfile

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime as dt

from google.colab import drive

# Mount google drive in colab
drive.mount('/content/gdrive/', force_remount=True)

# Unzip dataset file in colab
kaggle_dir = '/content/gdrive/MyDrive/Kaggle/'

filename = 'isic-2024-challenge.zip'

with zipfile.ZipFile(f'{kaggle_dir}/{filename}', 'r') as zp:
  zp.extractall('Kaggle-ISIC-Challenge/dataset/')

Mounted at /content/gdrive/


In [2]:
!nvidia-smi

Sun Aug 18 03:18:56 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0              45W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [3]:
train_csv = f'Kaggle-ISIC-Challenge/dataset/train-metadata.csv'
train_df = pd.read_csv(train_csv)

columns = train_df.columns

print(f'Columns: {columns}')

# TODO: Add all variables
# Continous columns- starting analysis on a subset of columns for now
continous = ['age_approx',
 'tbp_lv_C',
 'clin_size_long_diam_mm',
 'tbp_lv_A',
 'tbp_lv_Aext',
 'tbp_lv_B',
 'tbp_lv_Bext',
 'tbp_lv_C',
 'tbp_lv_Cext',
 'tbp_lv_H',
 'tbp_lv_Hext',
'tbp_lv_L',
'tbp_lv_Lext',
'tbp_lv_areaMM2',
'tbp_lv_area_perim_ratio',
'tbp_lv_color_std_mean',
'tbp_lv_deltaA',
'tbp_lv_deltaB',
'tbp_lv_deltaL',
'tbp_lv_deltaLB',
'tbp_lv_deltaLBnorm',
'tbp_lv_eccentricity',]

# Categorical columns
categorical = ['anatom_site_general',
 'tbp_tile_type',
 'image_type',
 'tbp_lv_location',
 'tbp_lv_location_simple']

continous_vars = train_df[continous]
categorical_vars = train_df[categorical]

continous_vars['target'] = train_df['target']
continous_vars['id'] = train_df['isic_id']


<ipython-input-3-a3300a721e99>:2: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv(train_csv)


Columns: Index(['isic_id', 'target', 'patient_id', 'age_approx', 'sex',
       'anatom_site_general', 'clin_size_long_diam_mm', 'image_type',
       'tbp_tile_type', 'tbp_lv_A', 'tbp_lv_Aext', 'tbp_lv_B', 'tbp_lv_Bext',
       'tbp_lv_C', 'tbp_lv_Cext', 'tbp_lv_H', 'tbp_lv_Hext', 'tbp_lv_L',
       'tbp_lv_Lext', 'tbp_lv_areaMM2', 'tbp_lv_area_perim_ratio',
       'tbp_lv_color_std_mean', 'tbp_lv_deltaA', 'tbp_lv_deltaB',
       'tbp_lv_deltaL', 'tbp_lv_deltaLB', 'tbp_lv_deltaLBnorm',
       'tbp_lv_eccentricity', 'tbp_lv_location', 'tbp_lv_location_simple',
       'tbp_lv_minorAxisMM', 'tbp_lv_nevi_confidence', 'tbp_lv_norm_border',
       'tbp_lv_norm_color', 'tbp_lv_perimeterMM',
       'tbp_lv_radial_color_std_max', 'tbp_lv_stdL', 'tbp_lv_stdLExt',
       'tbp_lv_symm_2axis', 'tbp_lv_symm_2axis_angle', 'tbp_lv_x', 'tbp_lv_y',
       'tbp_lv_z', 'attribution', 'copyright_license', 'lesion_id',
       'iddx_full', 'iddx_1', 'iddx_2', 'iddx_3', 'iddx_4', 'iddx_5',
       'mel_mitotic_

<ipython-input-3-a3300a721e99>:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  continous_vars['target'] = train_df['target']
<ipython-input-3-a3300a721e99>:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  continous_vars['id'] = train_df['isic_id']


In [4]:
from sklearn.model_selection import train_test_split

train_df = continous_vars.copy()
# Normalize columns
for col in continous:
  train_df[col] = (train_df[col] - train_df[col].min()) / (train_df[col].max() - train_df[col].min())

# Get NaN value counts age_approx : 2798
#nan_counts = continous_vars.isna().sum()
mean_age = train_df['age_approx'].mean()
train_df['age_approx'].fillna(mean_age, inplace=True)

# Split dataset
train, test = train_test_split(train_df, test_size=0.2, random_state=42)

print(f'Train shape: {train.shape}')
print(f'Test shape: {test.shape}')

Train shape: (320847, 24)
Test shape: (80212, 24)


In [5]:
print('Initial:', train['target'].value_counts())

def undersample_data(df, divisor = 5):
  # Undersample class 0 by 1/5 times
  cls_indices = [idx for idx, cls in enumerate(df['target'].values) if cls == 0]
  cls_1 = [idx for idx, cls in enumerate(df['target'].values) if cls == 1]
  to_keep = len(cls_indices)//5
  np.random.shuffle(cls_indices)
  indices_to_keep = cls_indices[:to_keep]
  all_indices = indices_to_keep + cls_1

  np.random.shuffle(all_indices)
  df = df.iloc[all_indices]
  return df

train = undersample_data(train)
test = undersample_data(test)

print('Final', train['target'].value_counts())

Initial: target
0    320528
1       319
Name: count, dtype: int64
Final target
0    64105
1      319
Name: count, dtype: int64


In [6]:
import tensorflow as tf
from PIL import Image

image_dir = 'Kaggle-ISIC-Challenge/dataset/train-image/image'
IMAGE_SIZE = 224
vgg_model = tf.keras.applications.vgg16.VGG16(include_top=False, weights = 'imagenet')
for layer in vgg_model.layers:
    layer.trainable = False

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


In [7]:
# Data Generator
from keras.layers import Flatten
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input

class DataGenerator(tf.keras.utils.Sequence):
  def __init__(self, image_ids, df, labels, batch_size=16):

    self.image_ids = image_ids
    self.df = df
    self.labels = labels
    self.batch_size = batch_size

    self.IMAGE_SIZE = 224
    self.image_dir = 'Kaggle-ISIC-Challenge/dataset/train-image/image'

  def __len__(self):
    return int(np.ceil(len(self.df) / self.batch_size))

  def __getitem__(self, index):

    batch_df = self.df[index * self.batch_size : (index + 1) * self.batch_size]

    # Image features
    batch_ids = self.image_ids[index * self.batch_size : (index + 1) * self.batch_size]

    batch_images = []
    image_features = []
    for id in batch_ids:
      q = Image.open(f'{image_dir}/{id}.jpg')
      q = np.array(q.resize((self.IMAGE_SIZE, self.IMAGE_SIZE)))
      features = vgg_model(np.array(q).reshape(1, self.IMAGE_SIZE, self.IMAGE_SIZE, 3))
      image_features.append(np.array(Flatten()(features)).reshape(7*7*512,))
      batch_images.append(q)

    length = len(batch_images)

    #image_features = tf.reshape(image_features, (length, -1))

    numerical_features = batch_df.to_numpy()

    # Batch labels
    batch_y = self.labels[index*self.batch_size : (index+1)*self.batch_size]

    return [np.array(image_features), numerical_features], batch_y

In [71]:
dtype= 'float32'
train_labels = train['target'].values
train_image_ids = train['id'].values
train_input = train.drop(['target', 'id'], axis = 1)

train_data_gen = DataGenerator(train_image_ids, train_input, train_labels, 1)

test_labels = test['target'].values
test_image_ids = test['id'].values
test_input = test.drop(['target', 'id'], axis = 1)

val_data_gen = DataGenerator(test_image_ids, test_input, test_labels,  1)

In [63]:
x, y = val_data_gen.__getitem__(0)

In [64]:
x.shape

(224, 224, 3)

In [10]:
from sys import stdlib_module_names
# Setting up a bayesian neural network

import tensorflow_probability as tfp
tfd = tfp.distributions
tfpl = tfp.layers

# A normal distribution (mean = 0, sd = 1) as prior for kernel weights
def prior(kernel_size: int, bias_size : int, dtype):
  '''
  To apply baye's theorem we start with a prior distribution which represents our prior beliefs about the weight distribution
  Prior distribution is not trainable
  '''
  n = kernel_size + bias_size
  return lambda t: tfd.Independent(tfd.Normal(loc = tf.zeros(n, dtype = dtype), scale = 1.), reinterpreted_batch_ndims = 1)

def random_gaussian_initializer(shape, dtype):
  '''
  Initialize random normal distributions for out surrogate posterior

  shape: number of (means + standard deviations)
  '''
  n = int(shape/2)
  loc_norm = tf.random_normal_initializer(mean = 0., stddev = 1.)
  loc = tf.Variable(initial_value = loc_norm(shape = (n,), dtype=dtype))

  scale_norm = tf.random_normal_initializer(mean = 2., stddev = 1.)
  scale = tf.Variable(initial_value = scale_norm(shape = (n,), dtype=dtype))

  return tf.concat([loc, scale], 0)


def posterior_mean_field(kernel_size: int, bias_size: int, dtype):
  '''
  Posterior distribution is a keras sequential model of two layers
  1. Layer 1 is made of 2*n trainable variables (representing mean and standard deviation for every kernel)
  2. Layer 2 is a tf probability distribution (normal distribution in this case) which is initalized using variables
  from the first layer

  This distribution represent our final beliefs for the weights of neural network
  '''
  n = kernel_size + bias_size
  c = np.log(np.expm1(1.))

  return tf.keras.Sequential([
      tfp.layers.VariableLayer(2 * n, dtype = dtype,
                    initializer = lambda shape, dtype: random_gaussian_initializer(shape, dtype), trainable = True),
      tfp.layers.DistributionLambda(lambda t: tfd.Independent(
          tfd.Normal(loc = t[..., :n], scale = 1e-5 + tf.nn.softplus(c + t[..., n:])), reinterpreted_batch_ndims = 1))
      ])


In [ ]:
x[0].shape

(8, 25088)

In [ ]:
train.shape[0]
examples = int(train.shape[0])

kl_divergence = (lambda q, p, _: tfd.kl_divergence(q, p) / tf.cast(tf.shape(q.sample())[0], tf.float32))


64424

In [36]:
x[0].shape

(8, 224, 224, 3)

In [74]:
type(y)#.shape

numpy.ndarray

In [ ]:
from keras.layers import Input, concatenate
from keras.models import Model

# Neural Network
#image_model_in = Input(x[0].shape)
#image_model = tfpl.DenseVariational(units = 1572, make_posterior_fn=posterior_mean_field, make_prior_fn=prior, kl_weight=1/examples, activation = 'gelu')(image_model_in)
#image_model = tfpl.DenseVariational(units = 512, make_posterior_fn=posterior_mean_field, make_prior_fn=prior, kl_weight=1/examples, activation = 'gelu')(image_model)
#image_model = tfpl.DenseVariational(units = 128, make_posterior_fn=posterior_mean_field, make_prior_fn=prior, kl_weight=1/examples, activation = 'gelu')(image_model)

numerical_model_in = Input(x[1].shape)
numerical_model = tfpl.DenseVariational(units = 128, make_posterior_fn=posterior_mean_field, make_prior_fn=prior, kl_weight=1/examples, activation = 'gelu')(numerical_model_in)
numerical_model = tfpl.DenseVariational(units = 64, make_posterior_fn=posterior_mean_field, make_prior_fn=prior, kl_weight=1/examples, activation = 'gelu')(numerical_model)

concat = concatenate([image_model, numerical_model])
ensemble_model_in = tfpl.DenseVariational(units = 128, make_posterior_fn=posterior_mean_field, make_prior_fn=prior, kl_weight=1/examples, activation = 'gelu')(concat)
ensemble_model_in = tfpl.DenseVariational(units = 32, make_posterior_fn=posterior_mean_field, make_prior_fn=prior, kl_weight=1/examples, activation = 'gelu')(ensemble_model_in)
ensemble_model_out = tfpl.DenseVariational(units = 1, make_posterior_fn=posterior_mean_field, make_prior_fn=prior, kl_weight=1/examples, activation = 'sigmoid')(ensemble_model_in)
ensemble_model_out = tfpl.DistributionLambda(lambda t: tfd.Bernoulli(logits = t))(ensemble_model_out)

ensemble_model = Model(inputs = [image_model_in, numerical_model_in], outputs = ensemble_model_out)

<ipython-input-139-894c8e518efa>:23: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  n = int(shape/2)


In [ ]:
print(tf.__version__)

2.17.0


In [ ]:
def nll_loss(y_true, y_pred):
  nll = -y_pred.log_prob(y_true)
  return nll

ensemble_model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.0002), loss = nll_loss, metrics = ['binary_accuracy'])
ensemble_model.fit(train_data_gen, validation_data = val_data_gen, epochs = 50, steps_per_epoch = 250)

Epoch 1/50
250/250 [==============================] - 277s 1s/step - loss: 1869.7915 - binary_accuracy: 0.3720 - val_loss: 1847.7811 - val_binary_accuracy: 0.3656
Epoch 2/50
250/250 [==============================] - 272s 1s/step - loss: 1826.3060 - binary_accuracy: 0.3690 - val_loss: 1804.9419 - val_binary_accuracy: 0.3718
Epoch 3/50
250/250 [==============================] - 259s 1s/step - loss: 1784.0408 - binary_accuracy: 0.3715 - val_loss: 1763.0695 - val_binary_accuracy: 0.3738
Epoch 4/50
250/250 [==============================] - 252s 1s/step - loss: 1742.5878 - binary_accuracy: 0.3490 - val_loss: 1722.0516 - val_binary_accuracy: 0.3716
Epoch 5/50
250/250 [==============================] - 252s 1s/step - loss: 1701.9110 - binary_accuracy: 0.3880 - val_loss: 1681.9110 - val_binary_accuracy: 0.3771
Epoch 6/50
250/250 [==============================] - 251s 1s/step - loss: 1662.2520 - binary_accuracy: 0.3735 - val_loss: 1642.6007 - val_binary_accuracy: 0.3743
Epoch 7/50
250/250 [==

In [ ]:
# Save model weights
ensemble_model.save_weights('/content/gdrive/MyDrive/Kaggle/ensemble_model_weights.h5')